In [308]:
from sympy import *
from sympy import lambdify, DiracDelta
import numpy as np
import plotly.graph_objects as go
from IPython.display import display, Math
from scipy.integrate import odeint

Введите свою функцию $W_р$(s) разомкнутой системы:

In [309]:
# Define open-loop transfer function W_open(s)
s = Symbol('s')

#↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓
W_open = (0.5*s+1) / ((0.3*s+1)*(0.04*s**2+0.02*s+1))
#↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑

# Display open-loop transfer function W_open(s)
display(Math('W_р(s) = ' + latex(W_open)))

# Print as LaTeX code for copy-paste to LaTeX document
print('Скопируйте код ниже и вставьте в свой LaTeX-файл:')
print('\\begin{equation}' + '\n' + '\t' + 'W_р(s) = ' +  latex(W_open) + '\n' + '\end{equation}')

<IPython.core.display.Math object>

Скопируйте код ниже и вставьте в свой LaTeX-файл:
\begin{equation}
	W_р(s) = \frac{0.5 s + 1}{\left(0.3 s + 1\right) \left(0.04 s^{2} + 0.02 s + 1\right)}
\end{equation}


Функция $W_з$(s) замкнутой системы:

In [310]:
# Define closed-loop transfer function W_closed(s)
Y_closed = (fraction(W_open)[0]).simplify()
U_closed = (fraction(W_open)[1] + fraction(W_open)[0]).simplify()
a = U_closed.coeff(s, degree(U_closed, s))
Y_closed = (Y_closed / a).simplify()
U_closed = (U_closed / a).simplify()
W_closed = Y_closed / U_closed

# Display closed-loop transfer function W_closed(s)
display(Math('W_з(s) = ' + latex(W_closed)))

# Print as LaTeX code for copy-paste to LaTeX document
print('Скопируйте код ниже и вставьте в свой LaTeX-файл:')
print('\\begin{equation}' + '\n' + '\t' + 'W_з(s) = ' +  latex(W_closed.simplify()) + '\n' + '\end{equation}')

<IPython.core.display.Math object>

Скопируйте код ниже и вставьте в свой LaTeX-файл:
\begin{equation}
	W_з(s) = \frac{41.6666666666667 s + 83.3333333333333}{1.0 s^{3} + 3.83333333333333 s^{2} + 68.3333333333333 s + 166.666666666667}
\end{equation}


Шаг по времени для моделирования:

In [311]:
Y_open = fraction(W_open)[0]
U_open = fraction(W_open)[1]
factors = factor_list(Y_open)[1] + factor_list(U_open)[1]
# Min of time constants
tau = min([pow(factor[0].coeff(s, degree(factor[0], s)), 1/degree(factor[0], s)) for factor in factors])
# Display time constant
display(Math('tau = ' + latex(tau)))

<IPython.core.display.Math object>

Область подобия:

In [312]:
# Find roots of U_closed(s)
roots = solve(U_closed, s)
# Find similarity region as 1/max|root|
similarity_region = 1/max([abs(root) for root in roots])
similarity_region

0.123963167584544

Составление системы дифференциальных уравнений из передаточной функции

In [313]:
# Define function y(t) and function u(t)
t = Symbol('t')
y = Function('y')(t)
u = Function('u')(t)

# Define system of differential equations from transfer function
def system_of_diff_eqs(W):
    # Define variables
    variables = []
    # Find Y(s) and U(s) from transfer function W(s)
    Y = fraction(W)[0].expand()
    U = fraction(W)[1].expand()
    # Define system of differential equations
    eqs = []
    # Define the order of U(s)
    n = degree(U, s)
    # Define x_1(t) = y(t)
    x_1 = Function('x_1')(t)
    # Add equations to the system
    for i in range(1, n):
        # Define x_i(t)
        x_i = Function('x_' + str(i))(t)
        # Define x_{i+1}(t)
        x_i_next = Function('x_' + str(i+1))(t)
        # Append new variable to the list of variables
        variables.append(x_i)
        # Append new equation to the system
        eqs.append(Eq(diff(x_i, t), -U.coeff(s, n-i)*x_1 + x_i_next + Y.coeff(s, n-i)*u)) 
    # Define x_n(t)
    x_n = Function('x_' + str(n))(t)
    # Append x_n(t) to the list of variables
    variables.append(x_n)
    # Append new equation to the system
    eqs.append(Eq(diff(x_n, t), -U.coeff(s, 0)*x_1 + Y.coeff(s, 0)*u))
    # Append last equation to the system
    eqs.append(Eq(y, x_1))
    # Return system of differential equations and list of variables
    return eqs, variables

In [314]:
# Find system of differential equations from transfer function W_closed(s)
eqs, variables = system_of_diff_eqs(W_closed)

# Display system of differential equations
latex_str = '\\begin{cases}'
for i in range(len(eqs)):
    latex_str += latex(eqs[i]) + '\\\\'
latex_str += '\\end{cases}'
display(Math(latex_str))

# Print latex code of system of differential equations
print('Скопируйте код ниже и вставьте в свой LaTeX-файл:')
print('\\begin{cases}')
for i in range(len(eqs)-1):
    print('\t'+latex(eqs[i]) + '\\\\')
print('\t'+latex(eqs[-1]))
print('\\end{cases}')

<IPython.core.display.Math object>

Скопируйте код ниже и вставьте в свой LaTeX-файл:
\begin{cases}
	\frac{d}{d t} x_{1}{\left(t \right)} = - 3.83333333333333 x_{1}{\left(t \right)} + x_{2}{\left(t \right)}\\
	\frac{d}{d t} x_{2}{\left(t \right)} = 41.6666666666667 u{\left(t \right)} - 68.3333333333333 x_{1}{\left(t \right)} + x_{3}{\left(t \right)}\\
	\frac{d}{d t} x_{3}{\left(t \right)} = 83.3333333333333 u{\left(t \right)} - 166.666666666667 x_{1}{\left(t \right)}\\
	y{\left(t \right)} = x_{1}{\left(t \right)}
\end{cases}


Определение констант

In [315]:
# Define quantization step
T = 0.01
time = 8
if float(tau) > 0.001:
    tau = 0.001
ts = np.arange(0, time, float(tau))
u0 = lambda t: 1 if t>=0 else 0

Решение системы дифференциальных уравнений для передаточной функции замкнутой системы:

In [316]:
# Solve system of differential equations eqs

# Functions of right side of system of differential equations using lambdify
func = lambdify((t, u, (variables)), [eq.rhs for eq in eqs[:-1]])

# Define function for solving system of differential equations
def f(y, t):
    return func(t, u0(t), y)

# RungeKutta4
def rungekutta4(f, y0, t, args=()):
    n = len(t)
    y = np.zeros((n, len(y0)))
    y[0] = y0
    for i in range(n - 1):
        h = t[i+1] - t[i]
        k1 = np.array(f(y[i], t[i], *args))
        k2 = np.array(f(y[i] + k1 * h / 2., t[i] + h / 2., *args))
        k3 = np.array(f(y[i] + k2 * h / 2., t[i] + h / 2., *args))
        k4 = np.array(f(y[i] + k3 * h, t[i] + h, *args))
        y[i+1] = y[i] + (h / 6.) * (k1 + 2*k2 + 2*k3 + k4)
    return y

# Find solution
true_solution = rungekutta4(f, np.zeros(len(variables)), ts)

In [317]:
# Plot solution using plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=ts, y=true_solution[:, 0], name='y(t)', line_shape='spline', line=dict(color='blue', width=3)))
fig.add_trace(go.Scatter(x=ts, y=[u0(t) for t in ts], name='u(t)', line_shape='spline', line=dict(color='red', width=3)))
fig.update_layout(xaxis_title='t', yaxis_title='y(t), u(t)')
fig.update_layout(template='plotly_white')
fig.update_layout(font=dict(size=14))
fig.update_layout(width=680, height=420,
                    margin=dict(l=25, r=25, b=25, t=25, pad=4))
fig.write_image('solution_first.pdf')
fig.show()

Система дифференциальных уравнений при добавлении в схему ключа:

In [318]:
u_star = Function('u^*')(t)
x_1_star = Function('x_1^*')(t)
eqs[-2] = eqs[-2].subs(u, u_star).subs(variables[0], x_1_star)

# Display system of differential equations
latex_str = '\\begin{cases}'
for i in range(len(eqs)):
    latex_str += latex(eqs[i]) + '\\\\'
latex_str += '\\end{cases}'
display(Math(latex_str))

    # if abs((t/T).round()*T - t) < tau/2:

# Print latex code of system of differential equations
print('Скопируйте код ниже и вставьте в свой LaTeX-файл:')
print('\\begin{cases}')
for i in range(len(eqs)-1):
    print('\t'+latex(eqs[i]) + '\\\\')
print('\t'+latex(eqs[-1]))
print('\\end{cases}')

<IPython.core.display.Math object>

Скопируйте код ниже и вставьте в свой LaTeX-файл:
\begin{cases}
	\frac{d}{d t} x_{1}{\left(t \right)} = - 3.83333333333333 x_{1}{\left(t \right)} + x_{2}{\left(t \right)}\\
	\frac{d}{d t} x_{2}{\left(t \right)} = 41.6666666666667 u{\left(t \right)} - 68.3333333333333 x_{1}{\left(t \right)} + x_{3}{\left(t \right)}\\
	\frac{d}{d t} x_{3}{\left(t \right)} = 83.3333333333333 u^{*}{\left(t \right)} - 166.666666666667 x^{*}_{1}{\left(t \right)}\\
	y{\left(t \right)} = x_{1}{\left(t \right)}
\end{cases}


In [319]:
eqs_new = eqs.copy()   
eqs_new[-2] = eqs_new[-2].subs(u_star, u).subs(x_1_star,variables[0])
    
func_key = lambdify((t, u, (variables)), [eq.rhs for eq in eqs_new[:-1]])

# Define function for solving system of differential equations
def f_key(y, t):
    ans = func_key(t, u0(t), y)
    if abs(floor(t/T+0.5)*T - t) < tau/2:
        ans[-1] *= 1/tau
    else:
        ans[-1] = 0
    return ans

# Find solution
solution = rungekutta4(f_key, np.zeros(len(variables)), ts)

In [320]:
# Plot solution using plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=ts, y=solution[:, 0], name='y(t)', line_shape='spline', line=dict(color='blue', width=3)))
fig.add_trace(go.Scatter(x=ts, y=[u0(t) for t in ts], name='u(t)', line_shape='spline', line=dict(color='red', width=3)))
fig.update_layout(xaxis_title='t', yaxis_title='y(t), u(t)')
fig.update_layout(template='plotly_white')
fig.update_layout(font=dict(size=14))
fig.update_layout(width=680, height=420,
                    margin=dict(l=25, r=25, b=25, t=25, pad=4))
fig.write_image(f'solution_second_{T}.pdf')
fig.show()

In [321]:
extr_0 = Function('Э^0')
# Apply function extr_0 to the x_1_star and u_star using replace
eqs_new = eqs.copy()
eqs_new[-2] = eqs_new[-2].subs(x_1_star, extr_0(x_1_star)).subs(u_star, extr_0(u_star))

# Display system of differential equations
latex_str = '\\begin{cases}'
for i in range(len(eqs_new)):
    latex_str += latex(eqs_new[i]) + '\\\\'
latex_str += '\\end{cases}'
display(Math(latex_str))

# Print latex code of system of differential equations
print('Скопируйте код ниже и вставьте в свой LaTeX-файл:')
print('\\begin{cases}')
for i in range(len(eqs_new)-1):
    print('\t'+latex(eqs_new[i]) + '\\\\')
print('\t'+latex(eqs_new[-1]))
print('\\end{cases}')

<IPython.core.display.Math object>

Скопируйте код ниже и вставьте в свой LaTeX-файл:
\begin{cases}
	\frac{d}{d t} x_{1}{\left(t \right)} = - 3.83333333333333 x_{1}{\left(t \right)} + x_{2}{\left(t \right)}\\
	\frac{d}{d t} x_{2}{\left(t \right)} = 41.6666666666667 u{\left(t \right)} - 68.3333333333333 x_{1}{\left(t \right)} + x_{3}{\left(t \right)}\\
	\frac{d}{d t} x_{3}{\left(t \right)} = 83.3333333333333 Э^{0}{\left(u^{*}{\left(t \right)} \right)} - 166.666666666667 Э^{0}{\left(x^{*}_{1}{\left(t \right)} \right)}\\
	y{\left(t \right)} = x_{1}{\left(t \right)}
\end{cases}


In [322]:
x_prev = 0
    
eqs_new = eqs.copy()
eqs_new[-2] = eqs_new[-2].subs(u_star, u).subs(x_1_star, variables[0])

func_extr_0_key = lambdify((t, u, (variables)), [eq.rhs for eq in eqs_new[:-1]], modules='numpy')

# Define function for solving system of differential equations
def f_extr_0_key(y, t):
    global x_prev
    ans = func_extr_0_key(t, u0(t), y)
    if abs(floor(t/T+0.5)*T - t) < tau/2:
        x_prev = ans[-1]
    else:
        ans[-1] = 0
    ans[-1] = x_prev
    return ans

def rungekutta4(f, y0, t, args=()):
    n = len(t)
    y = np.zeros((n, len(y0)))
    y[0] = y0
    for i in range(n - 1):
        h = t[i+1] - t[i]
        k1 = np.array(f(y[i], t[i], *args))
        k2 = np.array(f(y[i] + k1 * h / 2., t[i] + h / 2., *args))
        k3 = np.array(f(y[i] + k2 * h / 2., t[i] + h / 2., *args))
        k4 = np.array(f(y[i] + k3 * h, t[i] + h, *args))
        y[i+1] = y[i] + (h / 6.) * (k1 + 2*k2 + 2*k3 + k4)
    return y

# Find solution
solution = rungekutta4(f_extr_0_key, np.zeros(len(variables)), ts)

In [323]:
# Plot solution using plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=ts, y=solution[:, 0], name='y(t) extrapolator', line_shape='spline', line=dict(color='blue', width=3)))
fig.add_trace(go.Scatter(x=ts, y=[u0(t) for t in ts], name='u(t)', line_shape='spline', line=dict(color='red', width=3)))
fig.add_trace(go.Scatter(x=ts, y=true_solution[:, 0], name='y(t) continuous', line_shape='spline', line=dict(color='green', width=3)))
fig.update_layout(xaxis_title='t', yaxis_title='y(t), u(t)')
fig.update_layout(template='plotly_white')
fig.update_layout(font=dict(size=14))
fig.update_layout(width=680, height=420,
                    margin=dict(l=25, r=25, b=25, t=25, pad=4))
fig.write_image(f'solution_third_{T}.pdf')
fig.show()